In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib as plot

In [ ]:
import gzip
import pathlib
import shutil
import requests
def download_data(url, data_dir="data", unarchive=False, clobber=False):
    data_dir = pathlib.Path(data_dir)
    data_dir.mkdir(parents=True, exist_ok=True)
    local_filename = data_dir / url.split('/')[-1]
    if (local_filename.exists() and clobber) or not local_filename.exists():
        with requests.get(url, stream=True) as rstream:
            with local_filename.open("wb") as f:
                shutil.copyfileobj(rstream.raw, f)
    if unarchive:
        local_filename_unarchived = data_dir / local_filename.stem
        if (
            local_filename_unarchived.exists() and clobber
        ) and not local_filename_unarchived.exists():
            with gzip.open(local_filename, "rb") as fin:
                with local_filename_unarchived.open("wb") as fout:
                    shutil.copyfileobj(fin, fout)
        return str(local_filename_unarchived)
    return local_filename

In [ ]:
urls = [
    ("http://download.ecmwf.int/test-data/cfgrib/era5-levels-members.grib", False),
    ("https://psl.noaa.gov/thredds/fileServer/Datasets/noaa.oisst.v2/sst.mnmean.nc", False),
    (
        "http://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/NCAR/CESM2/historical/r11i1p1f1/Amon/tas/gn/v20190514/tas_Amon_CESM2_historical_r11i1p1f1_gn_200001-201412.nc",
        False,
    ),
    (
        "http://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/NCAR/CESM2/historical/r11i1p1f1/Amon/ta/gn/v20190514/ta_Amon_CESM2_historical_r11i1p1f1_gn_200001-201412.nc",
        False,
    ),
    (
        "http://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/NCAR/CESM2/historical/r11i1p1f1/Ofx/areacello/gr/v20190514/areacello_Ofx_CESM2_historical_r11i1p1f1_gr.nc",
        False,
    ),
    (
        "http://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/NCAR/CESM2/historical/r11i1p1f1/Omon/tos/gr/v20190514/tos_Omon_CESM2_historical_r11i1p1f1_gr_200001-201412.nc",
        False,
    ),
    (
        "http://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/NCAR/CESM2/historical/r9i1p1f1/Omon/tos/gr/v20190311/tos_Omon_CESM2_historical_r9i1p1f1_gr_200001-201412.nc",
        False,
    ),
    (
        "http://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/NCAR/CESM2/historical/r7i1p1f1/Omon/tos/gr/v20190311/tos_Omon_CESM2_historical_r7i1p1f1_gr_200001-201412.nc",
        False,
    ),
    (
        "http://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/NCAR/CESM2/historical/r8i1p1f1/Omon/tos/gr/v20190311/tos_Omon_CESM2_historical_r8i1p1f1_gr_200001-201412.nc",
        False,
    ),
]
for url, unarchive in urls:
    download_data(url, unarchive=unarchive)

In [ ]:
ds = xr.open_dataset("./data/tas_Amon_CESM2_historical_r11i1p1f1_gn_200001-201412.nc", engine="netcdf4")

In [ ]:
ds

In [ ]:
ds.info()

In [ ]:
# variables in the dataset
ds.data_vars

In [ ]:
# dataset dimensions
ds.dims

In [ ]:
# dataset coordinates
ds.coords

In [ ]:
# dataset global attributes
ds.attrs

In [ ]:
# Extract the tas variable (dataarray)
ds["tas"]

In [ ]:
# ds["tas"] is equivalent to ds.tas
ds.tas

In [ ]:
# The actual array data
ds["tas"].data

In [ ]:
# datarray coordinates
ds["tas"].coords

In [ ]:
# dataarray attributes
ds["tas"].attrs

In [ ]:
ds

In [ ]:
with xr.set_options(display_style="text"):
    print(ds)

In [ ]:
url = "http://crd-esgf-drc.ec.gc.ca/thredds/dodsC/esgD_dataroot/AR6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp126/r12i1p2f1/Amon/wap/gn/v20190429/wap_Amon_CanESM5_ssp126_r12i1p2f1_gn_201501-210012.nc"

In [ ]:
xr.open_dataset(url, engine="netcdf4", chunks={})

In [ ]:
temp = ds["tas"].data  # retrieve numpy array
temp

In [ ]:
temp.shape, temp.ndim

In [ ]:
temp[:, 20, 40]

In [ ]:
ds.tas.isel()  # the original object i.e. no selection

In [ ]:
ds.tas.isel(lat=100)

In [ ]:
ds.tas.isel(lat=100, time=[-2, -1])

In [ ]:
ds.tas.isel(lon=100, time=slice(10, 20))

In [ ]:
ds.tas.sel(time="2013")

In [ ]:
ds.tas.sel(time=slice("2013-01-01", "2014-12-31"))

In [ ]:
ds.tas.sel(lat=39.5, lon=105.7, method='nearest')

In [ ]:
ds.tas.sel(lat=slice(39, 39.5), lon=slice(106.1, 106.3))

In [ ]:
ds.tas.isel(time=0).sel(lon=slice(20, 160), lat=slice(-80, 25))

In [ ]:
ds.tas.interp(lat=[10, 10.1, 10.2], method='nearest')

In [ ]:
ds.tas.sel(lon=100, lat=10, method='nearest').plot(marker="o", size=6);

In [ ]:
ds.tas.isel(time=-10).sel(lon=slice(20, 160), lat=slice(-80, 25)).plot(robust=True, figsize=(8, 6));

In [ ]:
# define keyword arguments that are passed to matptolib.pyplot.colorbar
colorbar_kwargs = {
    "orientation": "horizontal",
    "label": "my clustom label",
    "pad": 0.2,
}

ds.tas.isel(lon=1).plot(
    x="time",  # coordinate to plot on the x-axis of the plot
    robust=True,  # set colorbar limits to 2nd and 98th percentile of data
    cbar_kwargs=colorbar_kwargs,
);

In [ ]:
ds.tas.sel(time=slice("2010", "2011")).plot(col="time", col_wrap=6, robust=True);

In [ ]:
ds.tas.plot();

In [ ]:
ds_air_all_pressure_levels = xr.open_dataset(
    "data/ta_Amon_CESM2_historical_r11i1p1f1_gn_200001-201412.nc", engine="netcdf4"
)
ds_air_all_pressure_levels

In [ ]:
data = ds_air_all_pressure_levels.ta.isel(time=-1).sel(lon=86.93, method='nearest')
fig = data.plot(size=6, yincrease=False)
fig.axes.set_title(
    f'Vertical profile of Temperature from pole to pole \nat longitude = {data.lon.data} and time = {data.time.data}',
    size=15,
);

In [ ]:
import hvplot.xarray

In [ ]:
ds.tas.hvplot()

In [ ]:
ds.tas.isel(time=1).hvplot(cmap="fire")

In [ ]:
ds.tas.isel(time=-1, lon=100).hvplot()

In [ ]:
ds.tas.sel(lat=28.5, lon=83.9, method='nearest').hvplot()

In [ ]:
ds.tas.hvplot(groupby="time", clim=(ds.tas.min(), ds.tas.max()), cmap='turbo')

In [ ]:
ds.tas.hvplot(
    groupby="time",
    clim=(ds.tas.min(), ds.tas.max()),
    cmap="turbo",
    widget_type="scrubber",
    widget_location="bottom",
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ds = xr.open_dataset("./data/tos_Omon_CESM2_historical_r11i1p1f1_gr_200001-201412.nc", engine="netcdf4")
ds

In [ ]:
ds.tos + 273.15

In [ ]:
ds.tos ** 2

In [ ]:
# Compute mean
ds.tos.mean()

In [ ]:
ds.tos.mean(dim='time').plot(size=7, robust=True);

In [ ]:
# compute temporal min
ds.tos.min(dim=['time'])

In [ ]:
# compute spatial sum
ds.tos.sum(dim=['lat', 'lon'])

In [ ]:
# compute temporal median
ds.tos.median(dim='time')

In [ ]:
ds.tos.groupby(ds.time.dt.month)

In [ ]:
ds.time.dt.year

In [ ]:
ds.time.dt.month

In [ ]:
ds.tos.groupby('time.month')

In [ ]:
tos_clim = ds.tos.groupby('time.month').mean()
tos_clim

In [ ]:
# Plot climatology at a specific point
tos_clim.sel(lon=310, lat=50, method='nearest').plot();

In [ ]:
# Plot zonal mean climatology
tos_clim.mean(dim='lon').transpose().plot.contourf(levels=12, robust=True, cmap='turbo');

In [ ]:
# Difference between January and December climatologies
(tos_clim.sel(month=1) - tos_clim.sel(month=12)).plot(size=6, robust=True);

In [ ]:
gb = ds.tos.groupby('time.month')
tos_anom = gb - gb.mean(dim='time')
tos_anom

In [ ]:
tos_anom.sel(lon=310, lat=50, method='nearest').plot();

In [ ]:
unweighted_mean_global_anom = tos_anom.mean(dim=['lat', 'lon'])
unweighted_mean_global_anom.plot();

In [ ]:
areacello = xr.open_dataset("data/areacello_Ofx_CESM2_historical_r11i1p1f1_gr.nc").areacello
areacello

In [ ]:
weighted_mean_global_anom = tos_anom.weighted(areacello).mean(dim=['lat', 'lon'])

In [ ]:
unweighted_mean_global_anom.plot(size=7)
weighted_mean_global_anom.plot()
plt.legend(['unweighted', 'weighted']);

In [ ]:
# resample to annual frequency
r = ds.tos.resample(time='AS')
r

In [ ]:
r.mean()

In [ ]:
# Compute a 5-month moving average
m_avg = ds.tos.rolling(time=5, center=True).mean()
m_avg

In [ ]:
lat = 50
lon = 310

m_avg.isel(lat=lat, lon=lon).plot(size=6)
ds.tos.isel(lat=lat, lon=lon).plot()
plt.legend(['5-month moving average', 'monthly data']);

In [ ]:
ds = xr.open_dataset("./data/tos_Omon_CESM2_historical_r11i1p1f1_gr_200001-201412.nc", engine="netcdf4")
ds

In [ ]:
sample = ds.tos.isel(time=-1)
sample

In [ ]:
masked_sample = sample.where(sample < 0.0)
masked_sample

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(19, 6))
sample.plot(ax=axes[0], robust=True)
masked_sample.plot(ax=axes[1], robust=True);

In [ ]:
sample.where((sample > 25) & (sample < 30)).plot(size=6, robust=True);

In [ ]:
sample.where((sample > 25) & (sample < 30), 0).plot(size=6, robust=True);

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
data = xr.open_dataset("./data/tos_Omon_CESM2_historical_r11i1p1f1_gr_200001-201412.nc", engine="netcdf4")
areacello = xr.open_dataset("./data/areacello_Ofx_CESM2_historical_r11i1p1f1_gr.nc", engine="netcdf4")

# Merge the two datasets in a single dataset
ds = xr.merge([data, areacello])
ds

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.Robinson(central_longitude=180))
ax.coastlines()
ax.gridlines()
ds.tos.isel(time=0).plot(
    robust=True, ax=ax, transform=ccrs.PlateCarree(), cbar_kwargs={'shrink': 0.5}
)
ax.set_global()

In [ ]:
tos_nino34 = ds.sel(lat=slice(-5, 5), lon=slice(190, 240))
tos_nino34

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.Robinson(central_longitude=180))
ax.coastlines()
ax.gridlines()
tos_nino34.tos.isel(time=0).plot(ax=ax, transform=ccrs.PlateCarree(), cbar_kwargs={'shrink': 0.5})
ax.set_extent((120, 300, 10, -10))

In [ ]:
gb = tos_nino34.tos.groupby('time.month')
tos_nino34_anom = gb - gb.mean(dim='time')
index_nino34 = tos_nino34_anom.weighted(tos_nino34.areacello).mean(dim=['lat', 'lon'])

In [ ]:
index_nino34_rolling_mean = index_nino34.rolling(time=5).mean()

In [ ]:
index_nino34.plot(size=8)
index_nino34_rolling_mean.plot()
plt.legend(['anomaly', '5-month running mean anomaly']);

In [ ]:
std_dev = tos_nino34.tos.std()
std_dev

In [ ]:
normalized_index_nino34_rolling_mean = index_nino34_rolling_mean / std_dev

In [ ]:
fig = plt.figure(figsize=(12, 6))

# Add -0.4/+0.4 lines which are the El Niño 3.4 thresholds
plt.fill_between(
    normalized_index_nino34_rolling_mean.time.data,
    normalized_index_nino34_rolling_mean.where(normalized_index_nino34_rolling_mean >= 0.4).data,
    0.4,
    color='red',
    alpha=0.9,
)
plt.fill_between(
    normalized_index_nino34_rolling_mean.time.data,
    normalized_index_nino34_rolling_mean.where(normalized_index_nino34_rolling_mean <= -0.4).data,
    -0.4,
    color='blue',
    alpha=0.9,
)

normalized_index_nino34_rolling_mean.plot(color='black')
plt.axhline(0, color='black', lw=0.5)
plt.axhline(0.4, color='black', linewidth=0.5, linestyle='dotted')
plt.axhline(-0.4, color='black', linewidth=0.5, linestyle='dotted');